#### Create and Train a RNN 
to predict the value of the voltage rise on the next period

Prediction is done in [VoltageRiseNum_Pred](VoltageRiseNum_Pred.ipynb)

We consider 

\begin{equation}
Z(k) = \begin{bmatrix} X(k-1)\\ \vdots \\ X(k-6)\ \end{bmatrix}, \;\; \text{where} \; \; 
X(k-i) = \begin{bmatrix} P_{load}(k-i) \\ P_{BT}(k-i) \\ P0013(k-i) \\ P0018(k-i) \\ P0100(k-i)\\ max\_vm\_pu(k-i) \end{bmatrix}
\end{equation}

to predict 

\begin{equation} Pred = \begin{bmatrix} P_{load}(k)\\  max\_vm\_pu(k) \end{bmatrix} \end{equation}



I am predicting column [0,5] i.e ['Cons','Voltage_rise'] even though the only column of interest is 5. 
This is because the RNN does not learn well when one is predicting on a unique feature. 
Column 5 is the one considered among all because it has the smoothest curve within the backward 
looking window considered hence yield the best accuracy

---

#### Import modules to be used

In [ ]:
#Import Modules 
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # Profiling 
import seaborn as sbn
import pickle, sys, importlib,  time
import os
from pickle import load
import tensorflow as tf
import joblib

#### Import Module for ML

In [ ]:
# import pakages forML
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

#### Import my own modules

In [8]:
# import psutil
function_folder = '../Modules/' 
# Add function folder to path if it is not already
if function_folder not in sys.path: sys.path.append(function_folder)

import oriFunctions as oriFc
from oriVariables import (network_folder, 
                          defAuth_hvBus_vRiseMax, 
                          defAuth_hvBus_vRiseMin, 
                          excel_folder, 
                          simResult_folder,
                          trainSet_end_M1, 
                          valSet_end_M2,
                          h_start_end, 
                          hm10_start_end, 
                          pd_Δt)

#### Import data Cleaned file for training the RNN

In [9]:
df_data = joblib.load(f'{simResult_folder}StLaurent_cleanedData.pkl')

# Extract only a part of data to be used as 
df_data = df_data[df_data.index<=valSet_end_M2]

In [11]:
# # Extract only the relavant testing set since the training set covers the first part of the data
df_final = df_data[df_data.index < trainSet_end_M1]

# Extract only dailight period i.e. from 07am to 7PM
# The daylight period is considered to be defined betwenn 07am and 7Pm excluded. 
per_index = df_final.index
per_daylight = (pd.Series(index=per_index.to_timestamp(), dtype=object)
                .between_time(*hm10_start_end) ).index.to_period(pd_Δt)

# Extract only daylight hours 
df_final = df_final.loc[per_daylight]


per_index = df_final.index
per_index2 = ( pd.Series(index=per_index.to_timestamp(), dtype=object
                           ).between_time(*h_start_end) ).index.to_period(pd_Δt)

#### Import the voltage rise from [VoltageRiseBinaryUpdated](VoltageRiseBinaryUpdated.ipynb)


In [12]:
# Import the voltage rise from 
numAndBin_vRise = joblib.load(simResult_folder+'Binary_Voltage_Rise.pkl')
df_final['Volt_Rise_bin'] = numAndBin_vRise['Volt_Rise_Bin']

### Set variables For numerical voltage rise prediction

In [13]:
# # Extract only the relavant testing set since the training set covers the first part of the data
df_final = df_data[df_data.index<trainSet_end_M1]
per_index = df_final.index
per_index2 = ( pd.Series(index=per_index.to_timestamp(), dtype=object
                        ).between_time(*h_start_end) ).index.to_period(pd_Δt)
df_final['Volt_Rise_num'] = numAndBin_vRise['known']

In [14]:
# # Separate training and testing set 
df_train = df_final[df_final.index<trainSet_end_M1]


# I'm using all the dataset to train the RNN to improve the performance since ive already
# tried with the validation  set and get an accuraccy of 94%
# # Separate training and testing set 
# df_train = df_final[df_final.index<trainSet_end_M1]

# Define scaler
numerical_scaler2 = MinMaxScaler()
numerical_scaler_out = MinMaxScaler()

numerical_scaler2.fit(df_train);
numerical_scaler_out.fit(df_train.iloc[:,[0,5]])

train_scaled = numerical_scaler2.transform(df_train)

##### Define Timeseries  generators


In [15]:
gen_length = 6 # 1 hour

batchSize = 24*7*2; #  (gen_length//6)  To convert in hour *24 hour * 7 days
# I am predicting column [0,5] i.e ['Cons','Voltage_rise'] even though the only collumn of interest is 5. 
# This is because the RNN does not learn well when one is predicting on a unique feature. 
# Column 5 is the one considered among all because it has the smoothest curve within the backward 
# looking window considered hence yield the best accuracy
train_generator = TimeseriesGenerator(train_scaled, train_scaled[:,[0,5]], 
                                      length = gen_length, 
                                      batch_size= batchSize )

# n_features = train_generator[0][0][0].shape[1]  # Define total number of features
n_features_inputs = 6  # Define total number of features in inputs 
n_features_outputs = 2  # Define total number of features to predicts

#### Define RNN

In [10]:
num_vRise_RNN = Sequential()

num_vRise_RNN.add( LSTM(units=128, activation='relu', input_shape=(gen_length,n_features_inputs)) )
# num_vRise_RNN.add( LSTM(units=128, activation='relu' ) )
num_vRise_RNN.add(Dense(units=n_features_outputs, activation='relu'))

num_vRise_RNN.compile(optimizer='adam', loss='mse', )

##### Define early stopping mechanism

In [11]:
early_stop = EarlyStopping(monitor= 'loss',patience=20, mode='min')

#### Train RNN

In [27]:
num_vRise_RNN.fit(train_generator, 
                  epochs=100, 
                  callbacks=[early_stop], 
                  )

Epoch 1/100
113/113 [==============================] - 2s 10ms/step - loss: 0.0173
Epoch 2/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0037
Epoch 3/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0026
Epoch 4/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0022
Epoch 5/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0020
Epoch 6/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0020
Epoch 7/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0019
Epoch 8/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0019
Epoch 9/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0019
Epoch 10/100
113/113 [==============================] - 1s 11ms/step - loss: 0.0018
Epoch 11/100
113/113 [==============================] - 1s 11ms/step - loss: 0.0017
Epoch 12/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0018
E

#### Save results

In [28]:
num_vRise_RNN.save(f'{network_folder}RNN/StLaurent_num_vRise_model')

INFO:tensorflow:Assets written to: pickle_files/RNN/StLaurent_num_vRise_model\assets


In [29]:
joblib.dump(numerical_scaler_out,f'{network_folder}RNN/StLaurent_num_vRise_scalerPred.plk')
joblib.dump(numerical_scaler2,f'{network_folder}RNN/StLaurent_num_vRise_scaler.plk')

['pickle_files/RNN/StLaurent_num_vRise_scaler.plk']